# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [10]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [11]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [12]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    if hyperparams['use_pca']:#Applico la pca
        if hyperparams.get('pca_standardize', False):
            s = StandardScaler()
            X_train = s.fit_transform(X_train)
            X_val = s.transform(X_val)

    n_components = hyperparams['pca_components']
    pca = PCA(n_components)
    X_train = pca.fit_transform(X_train)
    X_val = pca.transform(X_val)


    if hyperparams['data_standardize']:
        # ...
        s=StandardScaler()
        X_train=s.fit_transform(X_train)
        X_val=s.transform(X_val)

    # Aggiungo il termine costante 1 ai dati
    # ...
        vettore_di_uni_tr = np.ones((X_train.shape[0], 1))
        X_train = np.c_[vettore_di_uni_tr,X_train, ]

        vettore_di_uni_val = np.ones((X_val.shape[0], 1))
        X_val= np.c_[vettore_di_uni_val,X_val]

    # Calcolo regressione lineare e in caso faccio la regolarizzazione
    if hyperparams['use_regularization']:
        # ...
        lambda_reg = hyperparams['reg_lambda']
        I = np.eye(X_train.shape[1])#Creo la matrice identica per usare il valore di lambda
        I[0, 0] = 0 
        parametri = np.linalg.inv(X_train.T @ X_train + lambda_reg * I) @ X_train.T @ y_train#formula scaturita dalla soluzione lineare con regolarizzazione
    else:
        # ...
        parametri=np.linalg.inv(X_train.T@X_train)@X_train.T@y_train#formula scaturita dalla soluzione lineare senza regolarizzazione

    # Calcolo predizioni
    # ...
    y_predetti_train=X_train@parametri
    
    y_predetti_val=X_val@parametri

    # Calcola il MAE
    # ...
    MAE_train=np.mean(np.abs(y_train - y_predetti_train))
    MAE_val=np.mean(np.abs(y_val - y_predetti_val))
    #print(MAE_train)
    #print(MAE_val)

    return MAE_train,MAE_val


train_fraction = 0.8 
test_fraction = 0.2

num_campioni,dim =X.shape
train=int(num_campioni*train_fraction)

# Dividi il dataset in training e test set
# ...
X_train_sano = X[:train]  #train "sano" perchè dopo lo divido tra vero train e validation
X_test=X[train:]
y_train_sano = y[:train]  
y_test=y[train:]

# Dividi il training set in training set effettivo e validation set
# ...
num_campioni,dim =X_train_sano.shape
train=int(num_campioni*train_fraction)
X_train = X_train_sano[:train]  
X_val=X_train_sano[train:]
y_train = y_train_sano[:train]  
y_val=y_train_sano[train:]

# Trova la configurazione di iperparametri migliore
config_migliore = None
min_val = float('inf')#faccio valere il valore minimo come in valore + grande possibile(infinito)
min_train = None

for config in configs:
    #print(config)
    mae_train, mae_val = pipeline(X_train.copy(), y_train.copy(), X_val.copy(), y_val.copy(), config)
    if mae_val < min_val:
        min_val = mae_val
        min_train = mae_train
        config_migliore = config

def rialleno_con_mae(X_train_sano, y_train_sano, X_test, y_test, config):
    X_train = X_train_sano.copy()
    X_test = X_test.copy()

    # PCA con standardizzazione per PCA
    if config['use_pca']:
        if config.get('pca_standardize', False):
            scaler_pca = StandardScaler()
            X_train = scaler_pca.fit_transform(X_train)
            X_test = scaler_pca.transform(X_test)
        n_components=config['pca_components']
        pca = PCA(n_components)#proiezione fatta con la pca
        X_train = pca.fit_transform(X_train)
        X_test = pca.transform(X_test)

    # Standardizzazione generale (se non fatta da PCA)
    if config['data_standardize']:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    # Aggiunge il termine costante
    X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]

    if config['use_regularization']:
        lambda_reg = config['reg_lambda']
        I = np.eye(X_train.shape[1])
        I[0, 0] = 0
        parametri = np.linalg.inv(X_train.T @ X_train + lambda_reg * I) @ X_train.T @ y_train_sano
    else:
        parametri = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train_sano

    y_pred_test = X_test @ parametri
    mae_test = np.mean(np.abs(y_test - y_pred_test))
    return mae_test

# Calcola MAE sul test set
mae_test = rialleno_con_mae(X_train_sano, y_train_sano, X_test, y_test, config_migliore)

# Stampa i risultati
print("\nMiglior configurazione:")
print(config_migliore)
print("\nMAE train:")
print(min_train)
print("MAE validation:")
print(min_val)
print("MAE test:")
print(mae_test)


/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(



Miglior configurazione:
{'use_pca': True, 'pca_standardize': False, 'pca_components': 10, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}

MAE train:
0.6061022582434107
MAE validation:
0.5735159061512182
MAE test:
0.5481392209725904
